In [1]:
import h2o
import time
import seaborn
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from h2o.estimators.glm import H2OGeneralizedLinearEstimator as GLM
from h2o.estimators.gbm import H2OGradientBoostingEstimator as GBM
from h2o.estimators.random_forest import H2ORandomForestEstimator as RF

%matplotlib inline

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /home/am/anaconda3/envs/h2o/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfvwskrf7
  JVM stdout: /tmp/tmpfvwskrf7/h2o_am_started_from_python.out
  JVM stderr: /tmp/tmpfvwskrf7/h2o_am_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_am_5e0any
H2O cluster total nodes:,1
H2O cluster free memory:,6.957 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
wine_df = h2o.import_file("../Chapter08/winequality-red.csv", destination_frame="wine_df")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
wine_df.describe()

Rows:1599
Cols:12




,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,real,real,real,real,real,real,real,real,real,real,real,int
mins,4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4,3.0
mean,8.31963727329581,0.5278205128205128,0.2709756097560975,2.5388055034396495,0.08746654158849279,15.874921826141339,46.46779237023139,0.996746679174484,3.311113195747342,0.6581488430268919,10.422983114446529,5.6360225140712945
maxs,15.9,1.58,1.0,15.5,0.611,72.0,289.0,1.00369,4.01,2.0,14.9,8.0
sigma,1.7410963181276957,0.17905970415353537,0.1948011374053186,1.4099280595072798,0.047065302010090106,10.460156969809725,32.895324478299074,0.001887333953842544,0.15438646490354285,0.16950697959010996,1.0656675818473949,0.807569439734705
zeros,0,0,132,0,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5.0


In [5]:
features =  wine_df.columns[:-1]
print(features)
wine_df['quality'] = (wine_df['quality'] > 7).ifelse(1,0)
wine_df['quality'] = wine_df['quality'].asfactor()

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


In [6]:
train_df, valid_df, test_df = wine_df.split_frame(ratios=[0.6, 0.2], seed=133)

In [7]:
wine_df.head()

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,0
7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,0
7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,0
11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,0
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,0
7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,0
7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4,0
7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10,0
7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,0
7.5,0.5,0.36,6.1,0.071,17,102,0.9978,3.35,0.8,10.5,0


In [8]:
model_glm = GLM(model_id='wine_glm', family = 'binomial')
model_glm.train(training_frame= train_df, validation_frame=valid_df, y = 'quality', x=features)
print(model_glm)

glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  wine_glm


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.007317364461273153
RMSE: 0.08554159491892323
LogLoss: 0.029499544979614416
Null degrees of freedom: 974
Residual degrees of freedom: 963
Null deviance: 92.78211357225342
Residual deviance: 57.52411271024812
AIC: 81.52411271024812
AUC: 0.9622543950361944
pr_auc: 0.19963400979648407
Gini: 0.9245087900723887
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1997544524038773: 


,0,1,Error,Rate
0,964.0,3.0,0.0031,(3.0/967.0)
1,5.0,3.0,0.625,(5.0/8.0)
Total,969.0,6.0,0.0082,(8.0/975.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1997545,0.4285714,5.0
max f2,0.1997545,0.3947368,5.0
max f0point5,0.1997545,0.46875,5.0
max accuracy,0.2500808,0.9917949,3.0
max precision,0.2500808,0.5,3.0
max recall,0.0121073,1.0,111.0
max specificity,0.4984876,0.9989659,0.0
max absolute_mcc,0.1997545,0.4289870,5.0
max min_per_class_accuracy,0.0121073,0.8841779,111.0
max mean_per_class_accuracy,0.0121073,0.9420889,111.0


Gains/Lift Table: Avg response rate:  0.82 %, avg score:  0.82 %



...

metric,threshold,value,idx
max f1,0.0451519,0.25,17.0
max f2,0.0451519,0.3846154,17.0
max f0point5,0.0451519,0.1851852,17.0
max accuracy,0.4420668,0.9803922,0.0
max precision,0.1265588,0.1666667,5.0
max recall,0.0009383,1.0,108.0
max specificity,0.4420668,0.9966777,0.0
max absolute_mcc,0.0451519,0.2872856,17.0
max min_per_class_accuracy,0.0121073,0.8,37.0
max mean_per_class_accuracy,0.0121073,0.8418605,37.0


Gains/Lift Table: Avg response rate:  1.63 %, avg score:  1.10 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0130719,0.2007850,0.0,0.0,0.0,0.3297758,0.0,0.3297758,0.0,0.0,-100.0,-100.0
,2,0.0228758,0.1195805,20.4,8.7428571,0.3333333,0.1391871,0.1428571,0.2480949,0.2,0.2,1940.0000000,774.2857143
,3,0.0326797,0.0819314,0.0,6.12,0.0,0.1011772,0.1,0.2040196,0.0,0.2,-100.0,512.0
,4,0.0424837,0.0782332,0.0,4.7076923,0.0,0.0793470,0.0769231,0.1752490,0.0,0.2,-100.0,370.7692308
,5,0.0522876,0.0495749,0.0,3.8250000,0.0,0.0603511,0.0625,0.1537056,0.0,0.2,-100.0,282.5
,6,0.1013072,0.0170965,8.16,5.9225806,0.1333333,0.0310058,0.0967742,0.0943347,0.4,0.6,716.0,492.2580645
,7,0.1503268,0.0090784,4.08,5.3217391,0.0666667,0.0125548,0.0869565,0.0676674,0.2,0.8,308.0,432.1739130
,8,0.2026144,0.0049527,0.0,3.9483871,0.0,0.0063910,0.0645161,0.0518541,0.0,0.8,-100.0,294.8387097
,9,0.3006536,0.0019440,0.0,2.6608696,0.0,0.0030786,0.0434783,0.0359491,0.0,0.8,-100.0,166.0869565
,10,0.4019608,0.0007546,1.9741935,2.4878049,0.0322581,0.0012025,0.0406504,0.0271918,0.2,1.0,97.4193548,148.7804878



Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-12-10 23:35:47,0.000 sec,0,46.3910568,0.0475806
,2018-12-10 23:35:47,0.023 sec,1,39.4786281,0.0405123
,2018-12-10 23:35:47,0.026 sec,2,33.7455785,0.0346835
,2018-12-10 23:35:47,0.030 sec,3,30.9715167,0.0318906
,2018-12-10 23:35:47,0.032 sec,4,29.7996230,0.0307666
,2018-12-10 23:35:47,0.034 sec,5,29.2248184,0.0302714
,2018-12-10 23:35:47,0.036 sec,6,28.8419384,0.0300152
,2018-12-10 23:35:47,0.037 sec,7,28.7677743,0.0299927
,2018-12-10 23:35:47,0.040 sec,8,28.7620564,0.0299922


In [9]:
test_glm = model_glm.model_performance(test_df)
print(test_glm)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.017228193204603934
RMSE: 0.1312562120610066
LogLoss: 0.13988271775187358
Null degrees of freedom: 317
Residual degrees of freedom: 306
Null deviance: 53.187557984070224
Residual deviance: 88.96540849019598
AIC: 112.96540849019598
AUC: 0.6038338658146964
pr_auc: 0.03346490361472496
Gini: 0.2076677316293929
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.17042651179749857: 


,0,1,Error,Rate
0,308.0,5.0,0.016,(5.0/313.0)
1,4.0,1.0,0.8,(4.0/5.0)
Total,312.0,6.0,0.0283,(9.0/318.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1704265,0.1818182,5.0
max f2,0.1704265,0.1923077,5.0
max f0point5,0.1704265,0.1724138,5.0
max accuracy,0.4984876,0.9811321,0.0
max precision,0.1704265,0.1666667,5.0
max recall,0.0000002,1.0,253.0
max specificity,0.4984876,0.9968051,0.0
max absolute_mcc,0.1704265,0.1682606,5.0
max min_per_class_accuracy,0.0006228,0.6,109.0
max mean_per_class_accuracy,0.0006228,0.6226837,109.0


Gains/Lift Table: Avg response rate:  1.57 %, avg score:  1.20 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0125786,0.2682428,0.0,0.0,0.0,0.3888962,0.0,0.3888962,0.0,0.0,-100.0,-100.0
,2,0.0220126,0.1409895,21.2,9.0857143,0.3333333,0.1789215,0.1428571,0.2989070,0.2,0.2,2020.0,808.5714286
,3,0.0314465,0.0869027,0.0,6.36,0.0,0.1094954,0.1,0.2420835,0.0,0.2,-100.0,536.0
,4,0.0408805,0.0696413,0.0,4.8923077,0.0,0.0747444,0.0769231,0.2034668,0.0,0.2,-100.0,389.2307692
,5,0.0503145,0.0539497,0.0,3.975,0.0,0.0609205,0.0625,0.1767394,0.0,0.2,-100.0,297.5
,6,0.1006289,0.0207514,0.0,1.9875,0.0,0.0374268,0.03125,0.1070831,0.0,0.2,-100.0,98.75
,7,0.1509434,0.0072660,0.0,1.325,0.0,0.0124220,0.0208333,0.0755294,0.0,0.2,-100.0,32.5000000
,8,0.2012579,0.0033140,3.975,1.9875,0.0625,0.0050510,0.03125,0.0579098,0.2,0.4,297.5,98.75
,9,0.3018868,0.0011772,0.0,1.325,0.0,0.0020589,0.0208333,0.0392928,0.0,0.4,-100.0,32.5000000
,10,0.3993711,0.0003684,2.0516129,1.5023622,0.0322581,0.0006774,0.0236220,0.0298670,0.2,0.6,105.1612903,50.2362205


In [10]:
model_gbm = GBM(model_id='wine_gbm')
model_gbm.train(training_frame= train_df, validation_frame=valid_df, y = 'quality', x=features)
print(model_gbm)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  boston_gbm


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 1.7528245314956014e-05
RMSE: 0.004186674732404706
LogLoss: 0.0007428475623428082
Mean Per-Class Error: 0.0
AUC: 1.0
pr_auc: 0.875
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9407067152946088: 


,0,1,Error,Rate
0,967.0,0.0,0.0,(0.0/967.0)
1,0.0,8.0,0.0,(0.0/8.0)
Total,967.0,8.0,0.0,(0.0/975.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.9407067,1.0,7.0
max f2,0.9407067,1.0,7.0
max f0point5,0.9407067,1.0,7.0
max accuracy,0.9407067,1.0,7.0
max precision,0.9737563,1.0,0.0
max recall,0.9407067,1.0,7.0
max specificity,0.9737563,1.0,0.0
max absolute_mcc,0.9407067,1.0,7.0
max min_per_class_accuracy,0.9407067,1.0,7.0
max mean_per_class_accuracy,0.9407067,1.0,7.0


Gains/Lift Table: Avg response rate:  0.82 %, avg score:  0.82 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0130719,0.0292142,15.3000000,15.3000000,0.25,0.2687476,0.25,0.2687476,0.2,0.2,1430.0,1430.0
,2,0.0228758,0.0145284,20.4,17.4857143,0.3333333,0.0153164,0.2857143,0.1601343,0.2,0.4,1940.0000000,1648.5714286
,3,0.0326797,0.0085113,0.0,12.24,0.0,0.0111344,0.2,0.1154343,0.0,0.4,-100.0,1124.0
,4,0.0424837,0.0046967,0.0,9.4153846,0.0,0.0061533,0.1538462,0.0902156,0.0,0.4,-100.0,841.5384615
,5,0.0522876,0.0018704,0.0,7.6500000,0.0,0.0025121,0.125,0.0737712,0.0,0.4,-100.0,665.0
,6,0.1013072,0.0007635,4.08,5.9225806,0.0666667,0.0011501,0.0967742,0.0386320,0.2,0.6,308.0,492.2580645
,7,0.1503268,0.0003368,8.16,6.6521739,0.1333333,0.0004935,0.1086957,0.0261955,0.4,1.0,716.0,565.2173913
,8,0.2026144,0.0002668,0.0,4.9354839,0.0,0.0002900,0.0806452,0.0195102,0.0,1.0,-100.0,393.5483871
,9,0.3006536,0.0001669,0.0,3.3260870,0.0,0.0001979,0.0543478,0.0132127,0.0,1.0,-100.0,232.6086957
,10,0.4019608,0.0001405,0.0,2.4878049,0.0,0.0001503,0.0406504,0.0099205,0.0,1.0,-100.0,148.7804878



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
,2018-12-10 23:35:48,0.007 sec,0.0,0.0902098,0.0475806,0.5,0.0,1.0,0.9917949,0.1270396,0.0865847,0.5,0.0,1.0,0.9836601
,2018-12-10 23:35:48,0.093 sec,1.0,0.0833847,0.0298118,0.9864917,0.1509415,33.2386364,0.0256410,0.1251854,0.0752581,0.6820598,0.0912656,11.1272727,0.0392157
,2018-12-10 23:35:48,0.130 sec,2.0,0.0778991,0.0252048,0.9933428,0.5077050,40.625,0.0051282,0.1234236,0.0702562,0.7800664,0.0836383,15.3000000,0.0130719
,2018-12-10 23:35:48,0.157 sec,3.0,0.0732807,0.0222438,0.9984488,0.7851103,77.5568182,0.0020513,0.1220807,0.0658502,0.9631229,0.1420264,15.3000000,0.0457516
,2018-12-10 23:35:48,0.185 sec,4.0,0.0696617,0.0202054,0.9985134,0.7864261,85.3125,0.0020513,0.1211706,0.0636629,0.9627907,0.1461473,15.3000000,0.0424837
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2018-12-10 23:35:48,0.704 sec,46.0,0.0051054,0.0009204,1.0,0.875,97.5,0.0,0.1138550,0.0812571,0.9481728,0.1404564,15.3000000,0.0196078
,2018-12-10 23:35:48,0.715 sec,47.0,0.0049350,0.0008843,1.0,0.875,97.5,0.0,0.1138469,0.0819501,0.9475083,0.1387026,15.3000000,0.0196078
,2018-12-10 23:35:48,0.728 sec,48.0,0.0046037,0.0008229,1.0,0.875,97.5,0.0,0.1138770,0.0831508,0.9455150,0.1367249,15.3000000,0.0196078
,2018-12-10 23:35:48,0.738 sec,49.0,0.0044403,0.0007896,1.0,0.875,97.5,0.0,0.1139101,0.0838331,0.9461794,0.1366994,15.3000000,0.0196078



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sulphates,3.3108284,1.0,0.3232392
total sulfur dioxide,1.5800264,0.4772299,0.1542594
alcohol,1.0994868,0.3320881,0.1073439
chlorides,0.8164967,0.2466140,0.0797153
density,0.6784384,0.2049150,0.0662366
fixed acidity,0.6442806,0.1945980,0.0629017
citric acid,0.6002331,0.1812939,0.0586013
pH,0.5455816,0.1647870,0.0532656
volatile acidity,0.4583561,0.1384415,0.0447497
residual sugar,0.2734688,0.0825983,0.0266990


In [11]:
test_gbm = model_gbm.model_performance(test_df)
print(test_gbm)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.015718138327642
RMSE: 0.1253719997752369
LogLoss: 0.12803113765206706
Mean Per-Class Error: 0.23003194888178913
AUC: 0.7722044728434504
pr_auc: 0.032857432345781645
Gini: 0.5444089456869008
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0004738379817841345: 


,0,1,Error,Rate
0,275.0,38.0,0.1214,(38.0/313.0)
1,3.0,2.0,0.6,(3.0/5.0)
Total,278.0,40.0,0.1289,(41.0/318.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0004738,0.0888889,36.0
max f2,0.0001695,0.1801802,84.0
max f0point5,0.0004738,0.0606061,36.0
max accuracy,0.0400819,0.9811321,0.0
max precision,0.0004738,0.05,36.0
max recall,0.0001055,1.0,138.0
max specificity,0.0400819,0.9968051,0.0
max absolute_mcc,0.0001695,0.1436916,84.0
max min_per_class_accuracy,0.0001695,0.7220447,84.0
max mean_per_class_accuracy,0.0001055,0.7699681,138.0


Gains/Lift Table: Avg response rate:  1.57 %, avg score:  0.06 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0125786,0.0133530,0.0,0.0,0.0,0.0207121,0.0,0.0207121,0.0,0.0,-100.0,-100.0
,2,0.0220126,0.0053979,0.0,0.0,0.0,0.0083805,0.0,0.0154271,0.0,0.0,-100.0,-100.0
,3,0.0314465,0.0027198,0.0,0.0,0.0,0.0040918,0.0,0.0120265,0.0,0.0,-100.0,-100.0
,4,0.0408805,0.0019163,0.0,0.0,0.0,0.0024195,0.0,0.0098095,0.0,0.0,-100.0,-100.0
,5,0.0503145,0.0017833,0.0,0.0,0.0,0.0018135,0.0,0.0083102,0.0,0.0,-100.0,-100.0
,6,0.1037736,0.0006314,3.7411765,1.9272727,0.0588235,0.0011600,0.0303030,0.0046268,0.2,0.2,274.1176471,92.7272727
,7,0.1540881,0.0003899,3.975,2.5959184,0.0625,0.0004665,0.0408163,0.0032683,0.2,0.4,297.5,159.5918367
,8,0.2012579,0.0002886,0.0,1.9875,0.0,0.0003388,0.03125,0.0025817,0.0,0.4,-100.0,98.75
,9,0.3018868,0.0001622,3.975,2.65,0.0625,0.0001971,0.0416667,0.0017868,0.4,0.8,297.5,165.0
,10,0.3993711,0.0001221,0.0,2.0031496,0.0,0.0001395,0.0314961,0.0013847,0.0,0.8,-100.0,100.3149606


In [12]:
model_rf = RF(model_id='wine_rf')
model_rf.train(training_frame= train_df, validation_frame=valid_df, y = 'quality', x=features)
print(model_rf)

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  boston_rf


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.008894848806863922
RMSE: 0.09431250610000734
LogLoss: 0.08389391894356298
Mean Per-Class Error: 0.29453205791106507
AUC: 0.6283609100310238
pr_auc: 0.019539466164439814
Gini: 0.2567218200620476
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.10294113159179685: 


,0,1,Error,Rate
0,932.0,35.0,0.0362,(35.0/967.0)
1,6.0,2.0,0.75,(6.0/8.0)
Total,938.0,37.0,0.0421,(41.0/975.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1029411,0.0888889,33.0
max f2,0.1029411,0.1449275,33.0
max f0point5,0.1029411,0.0641026,33.0
max accuracy,0.3571429,0.9907692,0.0
max precision,0.1029411,0.0540541,33.0
max recall,0.0,1.0,399.0
max specificity,0.3571429,0.9989659,0.0
max absolute_mcc,0.0038342,0.1105805,100.0
max min_per_class_accuracy,0.0006778,0.625,160.0
max mean_per_class_accuracy,0.0006778,0.7054679,160.0


Gains/Lift Table: Avg response rate:  0.82 %, avg score:  1.04 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0130719,0.1013245,15.3000000,15.3000000,0.25,0.3057002,0.25,0.3057002,0.2,0.2,1430.0,1430.0
,2,0.0228758,0.0982296,0.0,8.7428571,0.0,0.1001619,0.1428571,0.2176124,0.0,0.2,-100.0,774.2857143
,3,0.0326797,0.0812170,20.4,12.24,0.3333333,0.0814128,0.2,0.1767525,0.2,0.4,1940.0000000,1124.0
,4,0.0424837,0.0764825,0.0,9.4153846,0.0,0.0802362,0.1538462,0.1544795,0.0,0.4,-100.0,841.5384615
,5,0.0555556,0.0612672,15.3000000,10.8,0.25,0.0614085,0.1764706,0.1325804,0.2,0.6,1430.0,980.0000000
,6,0.1045752,0.0402743,4.08,7.6500000,0.0666667,0.0499362,0.125,0.0938409,0.2,0.8,308.0,665.0
,7,0.1666667,0.0204480,0.0,4.8,0.0,0.0274341,0.0784314,0.0691011,0.0,0.8,-100.0,380.0
,8,0.2091503,0.0202446,4.7076923,4.78125,0.0769231,0.0203137,0.078125,0.0591912,0.2,1.0,370.7692308,378.125
,9,0.5196078,0.0005685,0.0,1.9245283,0.0,0.0020387,0.0314465,0.0250435,0.0,1.0,-100.0,92.4528302
,10,0.6307190,0.0004777,0.0,1.5854922,0.0,0.0004882,0.0259067,0.0207177,0.0,1.0,-100.0,58.5492228



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
,2018-12-10 23:35:49,0.003 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-12-10 23:35:49,0.027 sec,1.0,0.1299786,0.5057728,0.6511299,0.0020576,0.5015432,0.6816901,0.1714167,0.7418950,0.6289037,0.0595158,10.2,0.0294118
,2018-12-10 23:35:49,0.037 sec,2.0,0.1119881,0.3177221,0.4075362,0.0037626,0.0,0.9948097,0.1212287,0.1594928,0.7617940,0.1655364,18.36,0.0098039
,2018-12-10 23:35:49,0.051 sec,3.0,0.1116063,0.3115158,0.4,0.0051881,1.1284722,0.1541667,0.1158886,0.1584016,0.6458472,0.1226480,12.24,0.0294118
,2018-12-10 23:35:49,0.065 sec,4.0,0.1235080,0.4017905,0.4112474,0.0056886,0.0,0.1071864,0.1125197,0.1418199,0.8023256,0.3164272,20.4,0.0228758
---,---,---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2018-12-10 23:35:49,0.527 sec,46.0,0.0945856,0.0839800,0.6245476,0.0203099,0.0,0.0410256,0.1158550,0.0575179,0.9149502,0.0683542,15.3000000,0.0130719
,2018-12-10 23:35:49,0.537 sec,47.0,0.0945306,0.0837267,0.6219623,0.0194779,0.0,0.0420513,0.1160509,0.0577992,0.9129568,0.0670099,15.3000000,0.0130719
,2018-12-10 23:35:49,0.548 sec,48.0,0.0943826,0.0835723,0.6271329,0.0199042,0.0,0.0410256,0.1149300,0.0544560,0.9308970,0.0920791,15.3000000,0.0130719
,2018-12-10 23:35:49,0.559 sec,49.0,0.0943003,0.0836845,0.6273268,0.0197101,0.0,0.0420513,0.1144590,0.0533360,0.9382060,0.1112034,15.3000000,0.0130719



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sulphates,39.2206802,1.0,0.1577585
alcohol,31.4726219,0.8024497,0.1265933
density,23.1070290,0.5891542,0.0929441
total sulfur dioxide,22.6715393,0.5780506,0.0911924
residual sugar,22.4706402,0.5729284,0.0903843
fixed acidity,22.3236027,0.5691794,0.0897929
citric acid,19.1848221,0.4891507,0.0771677
free sulfur dioxide,17.9648457,0.4580452,0.0722605
volatile acidity,17.6737518,0.4506233,0.0710897
chlorides,16.5877552,0.4229339,0.0667214


In [13]:
test_rf = model_rf.model_performance(test_df)
print(test_rf)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.015312985031476338
RMSE: 0.1237456465152465
LogLoss: 0.07774923041444476
Mean Per-Class Error: 0.160702875399361
AUC: 0.7437699680511183
pr_auc: 0.06459019745180232
Gini: 0.48753993610223656
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.04150581955909727: 


,0,1,Error,Rate
0,294.0,19.0,0.0607,(19.0/313.0)
1,2.0,3.0,0.4,(2.0/5.0)
Total,296.0,22.0,0.066,(21.0/318.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0415058,0.2222222,18.0
max f2,0.0415058,0.3571429,18.0
max f0point5,0.0415058,0.1612903,18.0
max accuracy,0.1801205,0.9811321,0.0
max precision,0.0415058,0.1363636,18.0
max recall,0.0001483,1.0,73.0
max specificity,0.1801205,0.9968051,0.0
max absolute_mcc,0.0002034,0.3096956,72.0
max min_per_class_accuracy,0.0204146,0.8,34.0
max mean_per_class_accuracy,0.0204146,0.8392971,34.0


Gains/Lift Table: Avg response rate:  1.57 %, avg score:  0.92 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0125786,0.1168406,0.0,0.0,0.0,0.1401194,0.0,0.1401194,0.0,0.0,-100.0,-100.0
,2,0.0220126,0.0937437,0.0,0.0,0.0,0.1005021,0.0,0.1231406,0.0,0.0,-100.0,-100.0
,3,0.0314465,0.0808132,0.0,0.0,0.0,0.0814931,0.0,0.1106463,0.0,0.0,-100.0,-100.0
,4,0.0408805,0.0616656,0.0,0.0,0.0,0.0678829,0.0,0.1007778,0.0,0.0,-100.0,-100.0
,5,0.0503145,0.0602154,21.2,3.975,0.3333333,0.0611423,0.0625,0.0933462,0.2,0.2,2020.0,297.5
,6,0.1006289,0.0402057,7.95,5.9625,0.125,0.0419945,0.09375,0.0676703,0.4,0.6,695.0,496.2500000
,7,0.1509434,0.0202966,3.975,5.3,0.0625,0.0235696,0.0833333,0.0529701,0.2,0.8,297.5,430.0
,8,0.2012579,0.0016111,0.0,3.975,0.0,0.0166602,0.0625,0.0438926,0.0,0.8,-100.0,297.5
,9,0.4842767,0.0005685,0.0,1.6519481,0.0,0.0006007,0.0259740,0.0185921,0.0,0.8,-100.0,65.1948052
,10,0.5251572,0.0005134,0.0,1.5233533,0.0,0.0005134,0.0239521,0.0171848,0.0,0.8,-100.0,52.3353293


In [14]:
from h2o.grid.grid_search import H2OGridSearch as Grid

hyper_params = {'max_depth':[2,4,6,8,10,12,14,16]}

grid = Grid(model_gbm, hyper_params, grid_id='depth_grid')

grid.train(training_frame= train_df, validation_frame=valid_df, y = 'quality', x=features)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [15]:
print(grid)

    max_depth           model_ids              logloss
0           2  depth_grid_model_1  0.07040736021614663
1           4  depth_grid_model_2   0.0787918395099991
2           6  depth_grid_model_3  0.08387145119986193
3          14  depth_grid_model_7   0.0849816145272973
4          10  depth_grid_model_5   0.0849816950502771
5          16  depth_grid_model_8  0.08498174004732731
6          12  depth_grid_model_6  0.08498183111077036
7           8  depth_grid_model_4  0.08919355715963359



In [16]:
from h2o.automl import H2OAutoML as AutoML

aml = AutoML(max_models = 10, max_runtime_secs=100, seed=2)
aml.train(training_frame= train_df, validation_frame=valid_df, y = 'quality', x=features)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [17]:
print(aml.leaderboard)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_2_AutoML_20181210_233553,0.836543,0.0451932,0.35128,0.0899151,0.00808472
XGBoost_1_AutoML_20181210_233553,0.829498,0.043128,0.442154,0.0900821,0.00811478
GLM_grid_1_AutoML_20181210_233553_model_1,0.826202,0.0423037,0.346626,0.0902158,0.00813888
XGBoost_3_AutoML_20181210_233553,0.74037,0.0514577,0.296083,0.0908641,0.00825629
GBM_1_AutoML_20181210_233553,0.633984,0.133934,0.298281,0.127564,0.0162726
GBM_2_AutoML_20181210_233553,0.630752,0.0605161,0.308428,0.0922692,0.0085136
GBM_3_AutoML_20181210_233553,0.594752,0.0607896,0.316701,0.09239,0.00853592
GBM_4_AutoML_20181210_233553,0.528697,0.0610144,0.328594,0.0924088,0.00853939
StackedEnsemble_BestOfFamily_AutoML_20181210_233553,0.512797,0.0480753,0.383144,0.0902879,0.0081519
StackedEnsemble_AllModels_AutoML_20181210_233553,0.349276,0.0480757,0.5,0.0902604,0.00814694


In [18]:
model_gbm.confusion_matrix()

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9407067152946088: 


,0,1,Error,Rate
0,967.0,0.0,0.0,(0.0/967.0)
1,0.0,8.0,0.0,(0.0/8.0)
Total,967.0,8.0,0.0,(0.0/975.0)
